In [1]:
# import sys
# import os
# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

In [2]:
import numpy as np

from my_stuff import MufexKeys
import requests
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.enums import DynamicOrderSettingsArrays
from quantfreedom.enums import *
from quantfreedom.helper_funcs import dos_cart_product
from quantfreedom.nb_funcs.nb_strategies.nb_strategy import (
    nb_strat_bt_create_ind,
    nb_strat_evaluate,
    nb_strat_get_current_ind_settings,
    nb_strat_get_ind_set_str,
    nb_strat_get_total_ind_settings,
)
from quantfreedom.nb_funcs.nb_simulate import nb_sim_df_backtest
from quantfreedom.sim_order_records import sim_get_or
from quantfreedom.nb_funcs.nb_sim_order_records import nb_sim_get_or


mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

In [3]:
candles_np = mufex_main.get_candles(
    symbol="BTCUSDT", timeframe="5m", candles_to_dl=3000)

It took 00 mins and 00 seconds to download 3000 candles


In [4]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)
backtest_settings = BacktestSettings(
    divide_records_array_size_by=1,
    gains_pct_filter=-50,
    total_trade_filter=40,
    upside_filter=-np.inf,
)
dos_arrays = DynamicOrderSettingsArrays(
    entry_size_asset=np.array([0]),
    max_equity_risk_pct=np.array([6, 9, 12]),
    max_trades=np.array([10]),
    num_candles=np.array([100]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2, 4, 6]),
    sl_based_on_add_pct=np.array([0.1, 0.5, 1.0, 0.05]),
    sl_based_on_lookback=np.array([10, 30]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    sl_to_be_ze_type=np.array([0]),
    static_leverage=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.1, 0.5, 1.0, 2]),
    trail_sl_when_pct=np.array([0.2, 0.5, 1, 2]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    long_or_short=LongOrShortType.Long,
    logger_bool=True,
    pg_min_max_sl_bcb=PriceGetterType.Min,
    sl_to_be_bool=False,
    z_or_e_type=ZeroOrEntryType.Nothing,
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    tp_fee_type=TakeProfitFeeType.Limit,
    trail_sl_bool=True,
)
dos_cart_arrays = dos_cart_product(
    dos_arrays=dos_arrays,
)

ExchangeSettings(
    limit_fee_pct = 0.0003,
    max_leverage = 150.0,
    market_fee_pct = 0.0009,
    mmr_pct = 0.004,
    min_leverage = 1.0,
    max_asset_size = 100.0,
    min_asset_size = 0.001,
    asset_tick_step = 3,
    position_mode = 1,
    leverage_mode = 1,
    price_tick_step = 1,
    leverage_tick_step = 2,
)


In [10]:
strat_records_df = nb_sim_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_cart_arrays=dos_cart_arrays,
    evaluate=nb_strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=nb_strat_get_current_ind_settings,
    get_ind_set_str=nb_strat_get_ind_set_str,
    get_total_ind_settings=nb_strat_get_total_ind_settings,
    ind_creator=nb_strat_bt_create_ind,
    nb_logger_type=NBLoggerType.Pass,
    starting_equity=1000.0,
    static_os=static_os,
)
strat_records_df

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 6
Total order settings to test: 1,152
Total combinations of settings to test: 6,912
Total candles: 3,000
Total candles to test: 20,736,000


,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,to_the_upside,fees_paid,total_pnl,ending_eq
0,0,1,135.0,60,75,-41.56,44.44,-0.0000,2286.460,-415.6331,584.3669
1,0,2,61.0,26,35,-9.07,42.62,-0.0000,1098.780,-90.6577,909.3423
2,0,3,56.0,22,34,31.06,39.29,0.1354,1059.279,310.6365,1310.6365
3,0,6,83.0,28,55,-17.71,33.73,-0.0000,1399.641,-177.1256,822.8744
4,0,7,49.0,20,29,59.30,40.82,0.0000,942.518,592.9548,1592.9548
...,...,...,...,...,...,...,...,...,...,...,...
1351,3,1143,42.0,9,33,4.18,21.43,0.0011,1970.157,41.7630,1041.7630
1352,3,1144,61.0,14,47,-29.92,22.95,-0.0000,3076.424,-299.1618,700.8382
1353,3,1145,61.0,14,47,-29.92,22.95,-0.0000,3076.424,-299.1618,700.8382
1354,3,1146,60.0,14,46,-30.42,23.33,-0.2351,2869.202,-304.2205,695.7795


In [11]:
strat_records_df.sort_values(by='to_the_upside', ascending=False)

,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,to_the_upside,fees_paid,total_pnl,ending_eq
9,0,12,52.0,21,31,78.74,40.38,0.7914,948.473,787.3649,1787.3649
10,0,13,52.0,21,31,78.74,40.38,0.7914,948.473,787.3649,1787.3649
161,0,468,42.0,19,23,47.62,45.24,0.6214,470.924,476.2268,1476.2268
140,0,396,60.0,22,38,82.39,36.67,0.6210,2074.468,823.9098,1823.9098
142,0,398,60.0,22,38,82.39,36.67,0.6210,2074.468,823.9098,1823.9098
...,...,...,...,...,...,...,...,...,...,...,...
737,2,64,78.0,44,34,-30.86,56.41,-0.8807,467.214,-308.6002,691.3998
778,2,192,78.0,44,34,-30.86,56.41,-0.8807,467.214,-308.6002,691.3998
896,2,560,78.0,43,35,-46.48,55.13,-0.9123,719.771,-464.8172,535.1828
921,2,688,78.0,43,35,-46.48,55.13,-0.9123,719.771,-464.8172,535.1828


In [12]:
order_records_df, data = nb_sim_get_or(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_cart_arrays=dos_cart_arrays,
    evaluate=nb_strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=nb_strat_get_current_ind_settings,
    get_ind_set_str=nb_strat_get_ind_set_str,
    get_total_ind_settings=nb_strat_get_total_ind_settings,
    ind_creator=nb_strat_bt_create_ind,
    nb_logger_type=NBLoggerType.Pass,
    starting_equity=1000.0,
    static_os=static_os,
    ind_set_index=0,
    dos_index=12,
)

In [13]:
res = requests.post("http://127.0.0.1:7779/plot", json=data)

returned_data = res.json()

print(returned_data)

{'ok': True}
